# Unidad 5 - Actividad 1
# Materia: **Análisis de Algoritmos y Estructuras para Datos Masivos**
# Alumno: **Luis Fernando Izquierdo Berdugo**
# Fecha: **25 de Septiembre de 2024**

Considere los siguientes datos para los experimentos:
- REAL: Datos reales, puede usar listas-posteo-100.json o puede generarlos (vea Unidad 2). Leer la primera lista de posteo (tamaño ~41328 datos, es la etiquetada con "_url")
- SIN8: Datos sintéticos con diferencias aleatorias entre 1 y 8, $n=10^7$
- SIN64: Datos sintéticos con diferencias aleatorias entre 1 y 64, $n=10^7$
- SIN1024: Datos sintéticos con diferencias aleatorias entre 1 y 1024, $n=10^7$

1. Representación de Diferencias en Listas de Posteo:

- Calcule las diferencias entre entradas contiguas en cada lista de posteo. Estas serán la base para los pasos de compresión subsiguientes.

2. Compresión de Diferencias:

- Aplique las técnicas de compresión Elias-$\gamma$ y Elias-$\delta$ a las diferencias.
- Utilice las codificaciones inducidas por los algoritmos de búsqueda B1 y B2, basándose en los identificadores de las diferencias contiguas.

3. Comparación de Tiempos de Compresión y Descompresión:

- Mida y registre los tiempos de compresión y descompresión para cada conjunto de datos y método de codificación.
- Elabore un gráfico para comparar estos tiempos. Use diferentes colores o estilos de línea para distinguir entre métodos.

4. Comparación del Ratio de Compresión:

- Calcule y compare el ratio de compresión para cada conjunto de datos y método de codificación.
- Realice un gráfico para visualizar estos ratios, con los métodos de codificación en el eje horizontal y los ratios en el vertical.

5. Análisis de los Resultados Observados en las Gráficas:

- Analice detalladamente los resultados mostrados en los gráficos. Identifique tendencias, patrones y anomalías.
- Compare el desempeño de los métodos de codificación en términos de eficiencia de compresión y tiempo de procesamiento.
- Discuta las implicaciones de estos resultados. Por ejemplo, ¿qué método ofrece el mejor equilibrio entre tiempo de compresión y ratio de compresión?
- Considere cualquier factor externo o limitación que podría haber influido en los resultados.

Las comparaciones deberán realizarse mediante figuras y tablas que resuman la información.



## Inciso 1 - Lectura y cálculo de diferencias

In [40]:
import json

def openLists(route):
    with open(f'{route}') as file:
        for line in file:
            data = json.loads(line)
            key, values = data
            if key == '_url':
                return values

def opendiff(route):
    with open(f'{route}') as file:
        for line in file:
            data = json.loads(line)
        return data
    
_url = openLists('listas-posteo-100.json')
d8 = opendiff('diff-8.json')
d64 = opendiff('diff-64.json')
d1024 = opendiff('diff-1024.json')


In [41]:
def calcDiff(lista):
    diff = []
    for i in range(len(lista)-1):
        val = lista[i+1] - lista[i]
        diff.append(val)
    return diff

diffUrl = calcDiff(_url)
diff8 = calcDiff(d8)
diff64 = calcDiff(d64)
diff1024 = calcDiff(d1024)

## Inciso 2

In [42]:
import numpy as np

def eliasGammaE(num):
    binario = bin(num)[2:]
    length = len(binario)
    prefix_unario = "0" * (length-1)   
    return prefix_unario + binario

def eliasGammaD(num):
    zeroCount = num.find("1")
    binary_representation = num[zeroCount :]
    decoded_integer = int(binary_representation, 2)
    return decoded_integer


In [43]:
urlGammaEncoded = [eliasGammaE(diff) for diff in diffUrl]
urlGammaDecoded = [eliasGammaD(diff) for diff in urlGammaEncoded]

In [44]:
from math import floor, log, pow
def Binary_Representation_Without_MSB(x):
    binary = "{0:b}".format(int(x))
    binary_without_MSB = binary[1:]
    return binary_without_MSB

def eliasDeltaE(k):
    Gamma = eliasGammaE(1 + floor(log(k, 2)))
    binary_without_MSB = Binary_Representation_Without_MSB(k)
    return Gamma+binary_without_MSB

def eliasDeltaD(x):
	x = list(x) 
	L=0
	while True: 
		if not x[L] == '0': 
			break
		L= L + 1
	
	# Reading L more bits and dropping ALL 
	x=x[2*L+1:] 
	
	# Prepending with 1 in MSB 
	x.insert(0,'1') 
	x.reverse() 
	n=0
	
	# Converting binary to integer 
	for i in range(len(x)): 
		if x[i]=='1': 
			n=n+pow(2,i) 
	return int(n) 

In [45]:
urlDeltaEncoded = [eliasDeltaE(diff) for diff in diffUrl]
print(urlDeltaEncoded)
urlDeltaDecoded = [eliasDeltaD(diff) for diff in urlDeltaEncoded]
print(urlDeltaDecoded)

['1', '1', '1', '0100', '0100', '1', '1', '1', '1', '1', '1', '1', '1', '0101', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0100', '1', '1', '1', '0100', '1', '1', '1', '1', '1', '0100', '1', '1', '1', '1', '1', '1', '1', '1', '01110', '1', '1', '1', '1', '0100', '1', '1', '1', '0100', '1', '1', '1', '0100', '1', '1', '1', '0100', '1', '1', '0100', '1', '1', '1', '0100', '1', '1', '1', '1', '1', '0100', '0100', '1', '0101', '1', '1', '0101', '1', '1', '0100', '1', '1', '1', '0101', '1', '1', '0100', '1', '0100', '1', '0100', '1', '0101', '01111', '0100', '0100', '1', '1', '1', '1', '1', '1', '0100', '01100', '0100', '1', '1', '1', '1', '1', '1', '1', '1', '01101', '1', '1', '1', '0100', '1', '1', '0101', '0100', '0101', '1', '1', '1', '0101', '0100', '1', '1', '0100', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0100', '1', '1', '1', '1', '01100', '1', '1', '1', '0100', '0100', '1', '1', '1', '1', '0100', '0100', '0100', '1', '1', '0100', '0100', '1', '1', '1', '

In [46]:
from bitarray import bitarray

# Función para codificar usando bitarray
def binary_encoding(x, nbits):
    A = bitarray()
    sp = 0
    ep = (1 << nbits) - 1
    while sp < ep:
        mid = (sp + ep) // 2
        if x <= mid:
            A.append(0)
            ep = mid
        else:
            A.append(1)
            sp = mid + 1
    return A

# Función para decodificar usando bitarray
def binary_decoding(A, nbits):
    v = 0
    for i in range(nbits):
        if A[i]:
            v = (v << 1) | 1
        else:
            v <<= 1
    return v

# Prueba del código
for i in range(1, 31):
    s = binary_encoding(i, 5)
    d = binary_decoding(s, 5)
    print(f"i = {i}, s = {s}, d = {d}")
    assert d == i

i = 1, s = bitarray('00001'), d = 1
i = 2, s = bitarray('00010'), d = 2
i = 3, s = bitarray('00011'), d = 3
i = 4, s = bitarray('00100'), d = 4
i = 5, s = bitarray('00101'), d = 5
i = 6, s = bitarray('00110'), d = 6
i = 7, s = bitarray('00111'), d = 7
i = 8, s = bitarray('01000'), d = 8
i = 9, s = bitarray('01001'), d = 9
i = 10, s = bitarray('01010'), d = 10
i = 11, s = bitarray('01011'), d = 11
i = 12, s = bitarray('01100'), d = 12
i = 13, s = bitarray('01101'), d = 13
i = 14, s = bitarray('01110'), d = 14
i = 15, s = bitarray('01111'), d = 15
i = 16, s = bitarray('10000'), d = 16
i = 17, s = bitarray('10001'), d = 17
i = 18, s = bitarray('10010'), d = 18
i = 19, s = bitarray('10011'), d = 19
i = 20, s = bitarray('10100'), d = 20
i = 21, s = bitarray('10101'), d = 21
i = 22, s = bitarray('10110'), d = 22
i = 23, s = bitarray('10111'), d = 23
i = 24, s = bitarray('11000'), d = 24
i = 25, s = bitarray('11001'), d = 25
i = 26, s = bitarray('11010'), d = 26
i = 27, s = bitarray('11011'),

In [47]:
encode = [binary_encoding(diff, 5) for diff in diffUrl]
decode = [binary_decoding(diff, 5) for diff in encode]

In [48]:
class BitStream:
    def _init_(self):
        self.arr = []
        self.i = 0

    def write(self, b):
        self.arr.append(b)

    def read(self):
        b = self.arr[self.i]
        self.i += 1
        return b

def binary_search(A, target, start, end):
    sp = start
    ep = end - 1

    while sp <= ep:
        mid = (sp + ep) // 2
        if target <= mid:
            A[0] = 0
            ep = mid - 1
        else:
            A[0] = 1
            sp = mid + 1

    return A

def B_1_encode(A, element):
    i = 1
    j = 2

    # Find the range for binary search
    while element >= j:
        A[0] = 1
        i += 1
        j = 2 ** i
    A[0] = 0

    # Binary search in the found range
    return binary_search(A, element, 2 ** (i - 1), j)

In [49]:
def binary_decoding(A, nbits):

    v = 0
    for i in range(nbits):
        v = (v << 1) | A[i]
    return v

def B_1_decode(A):
    print(A)
    i = 0
    while A[i] == 1:
        i += 1
    inicio = 1 << i
    nbits = i
    return inicio + binary_decoding(A[i+1:], nbits)

In [63]:
class BitStream:
    def __init__(self):
        self.arr = []
        self.i = 0

    def write(self, b):
        self.arr.append(b)

    def read(self):
        b = self.arr[self.i]
        self.i += 1
        return b

def binary_search(A, target, inicio, final):
    sp = inicio
    ep = final - 1

    while sp <= ep:
        mid = (sp + ep) // 2
        if target <= mid:
            A.write(0)
            ep = mid - 1
        else:
            A.write(1)
            sp = mid + 1

    return A

def B_1_encode(A, elemento):
    i = 1
    j = 2

    # Find the range for binary search
    while elemento >= j:
        A.write(1)
        i += 1
        j = 2 ** i

    A.write(0)

    # Binary search in the found range
    return binary_search(A, elemento, 2 ** (i - 1), j)

def binary_decoding(A, nbits):
    v = 0
    for i in range(nbits):
        v = (v << 1) | A.read()
    return v

def B_1_decode(A):
    i = 0
    while A.read() == 1:
        i += 1
    inicio = 1 << i
    nbits = i
    return inicio + binary_decoding(A, nbits)


# Example usage
test = BitStream()
encode2 = [B_1_encode(test, diff) for diff in diffUrl]
decode2 = [B_1_decode(diff) for diff in encode2]

#print(encode2)
print(decode2)
print(diffUrl)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
